In [3]:
pip install --user selenium webdriver-manager beautifulsoup4 pandas


  Using cached selenium-4.35.0-py3-none-any.whl.metadata (7.4 kB)
  Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl.metadata (12 kB)
  Using cached trio-0.30.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
Using cached selenium-4.35.0-py3-none-any.whl (9.6 MB)
Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl (27 kB)
Using cached certifi-2025.8.3-py3-none-any.whl (161 kB)
Using cached trio-0.30.0-py3-none-any.whl (499 kB)
Using cached trio_websocket-0.12.2-py3-none-any.whl (21 kB)
Using cached attrs-25.3.0-py3-none-any.whl (63 kB)
Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install --user selenium webdriver-manager beautifulsoup4 pandas


In [5]:
"""from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time, random, re
import pandas as pd

options = Options()
options.add_argument("--headless")  
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("window-size=1400,600")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0 Safari/537.36")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

def extract_ram(text):
    m = re.search(r"(\d+)\s*GB\s*(RAM)?", text, re.I)
    if m: return int(m.group(1))
    m2 = re.search(r"(\d+)\s*GB\s*/\s*\d+\s*GB", text, re.I)
    return int(m2.group(1)) if m2 else None

def extract_storage(text):
    m = re.search(r"(\d+)\s*GB\s*(ROM|Storage)?", text, re.I)
    if m: return int(m.group(1))
    m2 = re.search(r"\d+\s*GB\s*/\s*(\d+)\s*GB", text, re.I)
    return int(m2.group(1)) if m2 else None

def extract_camera(text):
    m = re.search(r"(\d+)\s*MP", text, re.I)
    return int(m.group(1)) if m else None

def extract_battery(text):
    m = re.search(r"(\d{4,5})\s*mAh", text, re.I)
    return int(m.group(1)) if m else None

def extract_display(text):
    m = re.search(r"(\d+\.?\d*)\s*inch", text, re.I)
    return float(m.group(1)) if m else None

def extract_brand(name):
    return name.split()[0] if name else None

def clean_price(p):
    m = re.sub(r"[^\d]", "", p)
    return int(m) if m else None

def clean_discount(d):
    m = re.sub(r"[^\d]", "", d)
    return int(m) if m else None

def clean_rating(r):
    try: return float(r)
    except: return None

def clean_reviews(rv):
    m = re.sub(r"[^\d]", "", rv)
    return int(m) if m else None
# ---------------------------------------

brands = [
    "Samsung", "Redmi", "Realme", "POCO", "Vivo", "Oppo", "Motorola",
    "Infinix", "OnePlus", "Apple", "Tecno", "Nokia", "iQOO"
]

all_data = []

for brand in brands:
    print(f"\n🚀 Scraping brand: {brand}")
    base_url = f"https://www.flipkart.com/search?q=mobile+{brand}&page={{}}"

    for page in range(1, 50):  
        driver.get(base_url.format(page))
        time.sleep(random.uniform(3, 7))  

        # Scroll to bottom to load all products
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        # Parse HTML
        soup = BeautifulSoup(driver.page_source, "html.parser")

        names = soup.find_all("div", class_="KzDlHZ")  
        prices = soup.find_all("div", class_="Nx9bqj")  
        discounts = soup.find_all("div", class_="UkUFwK")  
        ratings = soup.find_all("div", class_="XQDdHH")  
        reviews = soup.find_all("span", class_="Wphh3N")  
        descs = soup.find_all("ul", class_="G4BRas")  

        if not names:  # no more products
            print(f"⚠️ No products found at page {page}, stopping {brand}.")
            break

        for i in range(len(names)):
            name = names[i].text.strip()
            desc = descs[i].text.strip() if i < len(descs) else ""

            price = prices[i].text.strip() if i < len(prices) else "N/A"
            discount = discounts[i].text.strip() if i < len(discounts) else "N/A"
            rating = ratings[i].text.strip() if i < len(ratings) else "N/A"
            review = reviews[i].text.strip() if i < len(reviews) else "N/A"

            all_data.append({
                "Name": name,
                "Brand": extract_brand(name),
                "Price": clean_price(price),
                "Discount_%": clean_discount(discount),
                "Rating": clean_rating(rating),
                "Reviews": clean_reviews(review),
                "RAM_GB": extract_ram(name) or extract_ram(desc),
                "Storage_GB": extract_storage(name) or extract_storage(desc),
                "Battery_mAh": extract_battery(desc),
                "Display_inches": extract_display(desc),
                "Camera_MP": extract_camera(desc),
                "Processor_Info": desc,
                "Category": "Mobile",
                "Search_Brand": brand,
                "Page": page
            })

        print(f"✅ Brand {brand} | Page {page} scraped | Total so far: {len(all_data)}")

df = pd.DataFrame(all_data)
df.drop_duplicates(subset=["Name", "Price"], inplace=True)

df.to_csv("flipkart_mobiles_multi_brand.csv", index=False, encoding="utf-8")

print(f"\n🎉 Scraping completed! Final products: {len(df)}")
driver.quit()"""



🚀 Scraping brand: Samsung
✅ Brand Samsung | Page 1 scraped | Total so far: 24
✅ Brand Samsung | Page 2 scraped | Total so far: 48
✅ Brand Samsung | Page 3 scraped | Total so far: 72
✅ Brand Samsung | Page 4 scraped | Total so far: 96
✅ Brand Samsung | Page 5 scraped | Total so far: 120
✅ Brand Samsung | Page 6 scraped | Total so far: 144
✅ Brand Samsung | Page 7 scraped | Total so far: 168
✅ Brand Samsung | Page 8 scraped | Total so far: 192
✅ Brand Samsung | Page 9 scraped | Total so far: 216
✅ Brand Samsung | Page 10 scraped | Total so far: 240
✅ Brand Samsung | Page 11 scraped | Total so far: 264
✅ Brand Samsung | Page 12 scraped | Total so far: 288
✅ Brand Samsung | Page 13 scraped | Total so far: 312
✅ Brand Samsung | Page 14 scraped | Total so far: 336
✅ Brand Samsung | Page 15 scraped | Total so far: 360
✅ Brand Samsung | Page 16 scraped | Total so far: 384
✅ Brand Samsung | Page 17 scraped | Total so far: 408
✅ Brand Samsung | Page 18 scraped | Total so far: 432
✅ Brand Samsun